In [4]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [8]:
dataset = []
for csv in Path('.').glob('*.tsv'):
    term = csv.name.replace('2022_12_11_inceldom_', '').replace('_references.tsv', '')
    if csv != 'usage_summary.tsv':
        csv = pd.read_csv(csv, sep='\t')
        csv['term'] = term
        dataset.append(csv)
dataset = pd.concat(dataset)
dataset = dataset.drop('Unnamed: 0', axis=1)

In [9]:
unique_users = dataset.groupby('term')['author'].nunique()
unique_users.name = "Unique Users Using Term" 
unique_threads = dataset.groupby('term')['URL'].nunique()
unique_threads.name = "Unique Threads Using Term"
unique_posts = dataset.groupby('term').size()
unique_posts.name = "Unique Posts Using Term"

usage_summary = pd.concat([unique_users, unique_threads, unique_posts], axis=1).reset_index()
tidy_names = {'rape': 'rape(s|ed)',
'raping': 'raping',
'sex_slave': 'sex slave(s|d)',
'sexual_slavery' : 'sexual slavery',
'sexual_assault': 'sexual assault(s|ing|ed)',
'sexual_assaulting': 'sexual assaulting',
'sexually_assaulting': 'sexually assaulting',
'harem': 'harem(s)'}
usage_summary = usage_summary.replace(tidy_names).set_index('term')
usage_summary.to_csv('usage_summary.tsv', sep='\t')

In [19]:
usage_summary.loc['rape(s|ed|ing)'] = usage_summary.loc['rape(s|ed)'] + usage_summary.loc['raping']
usage_summary.loc['sex(|ual) slave(s|d|ry)'] = usage_summary.loc['sex slave(s|d)'] + usage_summary.loc['sexual slavery']
usage_summary.loc['sexual(|lly) assualt(s|ing|ed)'] = usage_summary.loc['sexual assault(s|ing|ed)'] + usage_summary.loc['sexual assaulting'] + usage_summary.loc['sexually assaulting']
usage_summary = usage_summary.loc[['harem(s)', 'rape(s|ed|ing)', 'sex(|ual) slave(s|d|ry)', 'sexual(|lly) assualt(s|ing|ed)']]

In [21]:
usage_summary['References To'] = ['Harem', 'Rape', 'Sex Slavery', 'Sexual Assault'] 

In [23]:
usage_summary.to_csv('usage_summary_tidied.tsv', sep='\t')

In [ ]:
!cut -f 1 ../2022_12_11_inceldom_discussion_scrape/submissions/

In [26]:
ls ../2022_12_11_inceldom_discussion_scrape/

a.py
complete_submissions_index.txt
submissions/
user_data/


In [33]:
with open('../2022_12_11_inceldom_discussion_scrape/created_dates.txt') as fh:
    created_dates = pd.Series([int(x.strip()) for x in fh])
    
with open('../2022_12_11_inceldom_discussion_scrape/replies.txt') as fh:
    comments = pd.Series([x.strip() for x in fh])

In [39]:
max(created_dates)

1670883386

In [ ]:
comments = []
for scrape in tqdm(Path('../2022_12_11_inceldom_discussion_scrape/submissions/').glob('*.tsv'), total=234615):
    pd.read_csv('')

In [59]:
index = pd.read_csv('../2022_12_11_inceldom_discussion_scrape/complete_submissions_index.txt', sep='\t',names=['URL', 'Comment_Data', 'User_Data'])

users = set()
post_count = 0
for scrape in tqdm(Path('../2022_12_11_inceldom_discussion_scrape/submissions/').glob('*.tsv'), total=index.shape[0]):
    try:
        scrape_df = pd.read_csv(scrape, sep='\t', comment='#')
        users = users.union(set(scrape_df['author'].unique()))
        post_count += scrape_df.shape[0]
    except:
        continue

100%|████████████████████████████████████████████| 234615/234615 [05:42<00:00, 684.76it/s]


In [61]:
print(f"Total Users: {len(users)}")
print(f"Total Posts: {post_count}")
print(f"Total Threads: {index.shape[0]}")

Total Users: 11678
Total Posts: 4974826
Total Threads: 234615
